<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_Deep_Learning_Con_TernsorFlow_Machine-Learning_e_IA-/blob/master/scripts/tema09/04-stylenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat

In [0]:
pip install tensorflow==1.15

# Stylenet

In [0]:
import os
import scipy.io
import scipy.misc
import imageio
from skimage.transform import resize
from operator import mul
from functools import reduce
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

In [5]:
print(tf.__version__)

1.15.0


Ficheros de imágenes

In [0]:
!rm -r temp_output_*.jpg

In [0]:
ops.reset_default_graph()
original_image_file = "original_image.jpg"
style_image_file = "style_image.jpg"

In [0]:
vgg_path = "imagenet-vgg-verydeep-19.mat"
original_image_weight = 5.0
style_image_weight = 500.0
regularization_weight = 100
learning_rate = 10
generations = 1000
output_generations = 100
beta1 = 0.9
beta2 = 0.999

In [0]:
original_image = imageio.imread(original_image_file)
style_image = imageio.imread(style_image_file)

In [0]:
target_shape = original_image.shape
style_image = resize(style_image, target_shape)

Redes neuronales del paper VGG19 disponible en [Arxiv.org](https://arxiv.org/pdf/1508.06576.pdf)

In [0]:
vgg_layers = ['conv1_1', 'relu1_1',
              'conv1_2', 'relu1_2', 'pool1',
              'conv2_1', 'relu2_1',
              'conv2_2', 'relu2_2', 'pool2',
              'conv3_1', 'relu3_1',
              'conv3_2', 'relu3_2',
              'conv3_3', 'relu3_3',
              'conv3_4', 'relu3_4', 'pool3',
              'conv4_1', 'relu4_1',
              'conv4_2', 'relu4_2',
              'conv4_3', 'relu4_3',
              'conv4_4', 'relu4_4', 'pool4',
              'conv5_1', 'relu5_1',
              'conv5_2', 'relu5_2',
              'conv5_3', 'relu5_3',
              'conv5_4', 'relu5_4']

In [0]:
def extract_net_info(path_to_mat_file):
    vgg_data = scipy.io.loadmat(path_to_mat_file)
    normalization_matrix = vgg_data["normalization"][0][0][0]
    mat_mean = np.mean(normalization_matrix, axis=(0,1))
    network_weights = vgg_data['layers'][0]
    return mat_mean, network_weights

In [0]:
def vgg_network(network_weights, init_image):
    network = {}
    image = init_image

    for i, layer in enumerate(vgg_layers):
        if layer[0] == 'c': #convolución
            weights, bias = network_weights[i][0][0][0][0]
            weights = np.transpose(weights, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            conv_layer = tf.nn.conv2d(image, tf.constant(weights), (1, 1, 1, 1), 'SAME')
            image = tf.nn.bias_add(conv_layer, bias)
        elif layer[0] == 'r': #relu
            image = tf.nn.relu(image)
        else:  #max pooling
            image = tf.nn.max_pool(image, (1, 2, 2, 1), (1, 2, 2, 1), 'SAME')
        network[layer] = image
    return network

In [0]:
original_layers = ['relu4_2', 'relu5_2']
style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']

In [0]:
# Get network parameters
normalization_mean, network_weights = extract_net_info(vgg_path)

In [0]:
shape = (1,) + original_image.shape
style_shape = (1,) + style_image.shape

In [0]:
original_features = {}
style_features = {}

In [97]:
style_weights = {l: 1./(len(style_layers)) for l in style_layers}
style_weights

{'relu1_1': 0.2,
 'relu2_1': 0.2,
 'relu3_1': 0.2,
 'relu4_1': 0.2,
 'relu5_1': 0.2}

In [0]:
g_original = tf.Graph()
with g_original.as_default(), tf.Session() as session1:
    image = tf.placeholder("float", shape=shape)
    vgg_net = vgg_network(network_weights, image)
    original_minus_mean = original_image - normalization_mean
    original_norm = np.array([original_minus_mean])
    for layer in original_layers:
        original_features[layer] = vgg_net[layer].eval(feed_dict={image:original_norm})

In [0]:
g_style = tf.Graph()
with g_style.as_default(), tf.Session() as session2:
    image = tf.placeholder("float", shape=style_shape)
    vgg_net = vgg_network(network_weights, image)
    style_minus_mean = style_image - normalization_mean
    style_norm = np.array([style_minus_mean])
    for layer in style_layers:
        features = vgg_net[layer].eval(feed_dict={image:style_norm})
        features = np.reshape(features, (-1, features.shape[3]))
        gram = np.matmul(features.T, features)/features.size
        style_features[layer] = gram

### Generarar la modificacion de la imagen

In [0]:
def modelo_final():
  with tf.Graph().as_default():
    
    initial = tf.random_normal(shape)*0.256
    init_image = tf.Variable(initial)
    vgg_net = vgg_network(network_weights, init_image)
    
    original_layers_w = {"relu4_2":0.5, "relu5_2":0.5}
    original_loss = 0
    for layer in original_layers:
        temp_original_loss = original_layers_w[layer]*original_image_weight *\
            (2*tf.nn.l2_loss(vgg_net[layer]-original_features[layer]))
        original_loss += temp_original_loss/original_features[layer].size
    style_loss = 0
    style_losses = []
    for style_layer in style_layers:
        layer = vgg_net[style_layer]
        feats, height, width, channels = [x.value for x in layer.get_shape()]
        size = height * width * channels
        features = tf.reshape(layer, (-1, channels))
        style_gram_matrix = tf.matmul(tf.transpose(features), features) / size
        style_expected = style_features[style_layer]
        style_losses.append(style_weights[style_layer] * 2 *
                            tf.nn.l2_loss(style_gram_matrix - style_expected) /
                            style_expected.size)
    style_loss += style_image_weight * tf.reduce_sum(style_losses)
    
    total_var_x = reduce(mul, init_image[:,1:,:,:].get_shape().as_list(),1)
    total_var_y = reduce(mul, init_image[:,:,1:,:].get_shape().as_list(),1)
    
    first_term = regularization_weight*2
    second_term_num = tf.nn.l2_loss(init_image[:,1:, :,:]- init_image[:,:shape[1]-1,:,:])
    second_term = second_term_num/total_var_y
    third_term_num = tf.nn.l2_loss(init_image[:,:,1:,:]-init_image[:,:,:shape[2]-1,:])
    third_term = third_term_num/total_var_x
    total_var_loss = first_term*(second_term+third_term)
    
    loss = original_loss+style_loss+total_var_loss
    
    optim = tf.train.AdamOptimizer(learning_rate, beta1, beta2)
    train_step = optim.minimize(loss)
    
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        for i in range(generations):
            train_step.run()
            
            if (i+1)% output_generations==0:
                print("Iteración {} de {}, loss {}".format(i+1, generations, session.run(loss)))
                image_eval = init_image.eval()
                best_image_add_mean = image_eval.reshape(shape[1:])+normalization_mean
                output_file = 'temp_output_{}.jpg'.format(i+1)
                imageio.imwrite(output_file, best_image_add_mean.astype(np.uint8))
        
        image_eval = init_image.eval()
        best_image_add_mean = image_eval.reshape(shape[1:])+normalization_mean
        output_file = 'final_output.jpg'
        imageio.imwrite(output_file, best_image_add_mean.astype(np.uint8))
  

# SISTEMA PARA USAR GPU DE GOOGLE COLAB

## imprimir el modelo

In [0]:
modelo.summary()

## ver el tiempo

In [0]:
import io

In [70]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    print('trabajando')
    #modelo.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=2,verbose=1))
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

print('tiempo de entrenamiento '+ str(cpu_time)+ ' segundos')

trabajando
tiempo de entrenamiento 0.0003107990000899008 segundos


## GPU verificacion disponibilidad


In [101]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SyntaxError('Gpu no encongrada')
print('GPU es : {}'.format(nombre_gpu))

GPU es : /device:GPU:0


#Entrenamiento con GPU

In [102]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    modelo_final()
    print('ejecutado')
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')



Iteración 100 de 1000, loss 6972318.0
Iteración 200 de 1000, loss 7299189.5
Iteración 300 de 1000, loss 5740503.5
Iteración 400 de 1000, loss 6013201.0
Iteración 500 de 1000, loss 5592290.5
Iteración 600 de 1000, loss 5536367.5
Iteración 700 de 1000, loss 5434729.5
Iteración 800 de 1000, loss 5107514.0
Iteración 900 de 1000, loss 5031511.0
Iteración 1000 de 1000, loss 5011045.5
ejecutado


In [82]:
print('tiempo de entrenamiento '+ str(gpu_time)+ ' segundos')

tiempo de entrenamiento 735.2419078980001 segundos


#USO DE TPU

Primero configuramos el note book con la divece TPU

## configuracion inicial de TPU


In [0]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
  print('TPU encontrada  ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR TPU no encontrada')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
def TPU_model():
  with tpu_strategy.scope():
    #------aqui colocamos nuestro modelo-----------
    modelo_final()

In [0]:
import timeit

def entrenamiento_tpu():
  TPU_model()
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')
